In [1]:

import tensorflow as tf
import os
import numpy as np
import random
import cv2 as cv




import matplotlib.pyplot as plt

##from skimage.io import imread,imshow

# model = tf.keras.models.load_model('Modelv1',compile=False)

# data = np.load('data.npz')
# X_train = data['X_train']
# X_test = data['X_test']
# Y_test = data['Y_test']

# preds_test = model.predict(X_test)
#preds_test = (preds_test > 0.5).astype(np.uint8)












2023-04-21 20:10:34.721200: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
preds_test = (preds_test > 0.5).astype(np.uint8)
1 in preds_test[0]

NameError: name 'preds_test' is not defined

In [ ]:
ix = random.randint(0, len(X_test))
img = cv.cvtColor(X_test[1],cv.COLOR_BGR2GRAY)
img2=np.array(preds_test[1]).astype('uint8')

# img2 = img2.transpose(2,0,1).reshape(128,-1)


for i in range(img2.shape[0]):
    for j in range(img2.shape[1]):
        if img2[i][j] == 1:
            img2[i][j] = 255


cv.imshow('X',img)
cv.waitKey(0)
# cv.imshow('Y',np.array(Y_test[ix]).astype('uint8'))
cv.imshow('Pred',img2)





cv.waitKey(0)
cv.destroyAllWindows()



NameError: name 'X_test' is not defined

In [6]:
img3 = cv.imread('/home/omkar/FireClassifier/segmented_imgs/img/img75/Fire34.jpg')
img4 = img3[:,:,2]
img5 = cv.blur(img4,(10,10))
contours, hierarchy = cv.findContours(img4,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
contours2, hierarchy = cv.findContours(img5,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
print(len(contours),len(contours2))

401 17


In [7]:
c1 = contours2
# c = list(c[5])
# c_sort_by_y = sorted(c,key = lambda x : x[0][1])
# c_sort_by_x = sorted(c,key = lambda x : x[0][0])
# c_sort_by_y
c_selected = c1[0]
for i in c1:
    if len(c_selected)< len(i): 
        c_selected = i

c = list(c_selected)
c_sort_by_y = sorted(c,key = lambda x : x[0][1])
c_sort_by_x = sorted(c,key = lambda x : x[0][0])
print(cv.contourArea(c_selected),len(c_selected))

188910.0 2078


In [9]:

import math

def contourprep(img):
    img = img[:,:,2]
    img = cv.blur(img,(10,10))
    contours, hierarchy = cv.findContours(img,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    c_selected = contours[0]
    for i in contours:
        if len(c_selected)< len(i): 
            c_selected = i

    c = list(c_selected)
    c_sort_by_y = sorted(c,key = lambda x : x[0][1])
    c_sort_by_x = sorted(c,key = lambda x : x[0][0])
    return contours,c_sort_by_x,c_sort_by_y
    
    

    
def getBaseWidthAndHeight(arr:list):
    min_y = arr[len(arr)-1][0][1]
    max_y = arr[0][0][1]
    range = (min_y - max_y)/4
    min_x = arr[len(arr)-1][0][0]
    max_x = arr[len(arr)-1][0][0]
    #print('hehe',max_x,min_x,range,max_y,min_y,min_y-range)

    
    i = len(arr)-1
    while i >= 0:
        if(arr[i][0][1] < int(min_y-range)):
            break
        if(arr[i][0][0] > max_x):
            max_x = arr[i][0][0]
        if(arr[i][0][0] < min_x):
            min_x = arr[i][0][0]
        i = i -1
    midpoint = math.floor((min_x + max_x)/2)

    return {"width":max_x - min_x,"height":min_y-max_y,"base_midpoint":midpoint}



def getSymmetryScore(arr:list,base_midpoint):
    

    min_x = arr[len(arr)-1][0][0]
    max_x = arr[0][0][0]
    mean = base_midpoint
    left_area = 0
    right_area = 0

    for i in range(0,len(arr)):
        if arr[i][0][0] < mean:
            left_area += mean-arr[i][0][0]
        else:
            right_area += arr[i][0][0]-mean
    return abs(left_area-right_area)/(left_area+right_area)

def getAttrributes(img):
    h_mat = img[:,:,0]
    s_mat = img[:,:,1]
    v_mat = img[:,:,2]
    number_of_nonblackpixels = np.count_nonzero(v_mat)
    average_h = h_mat[np.nonzero(h_mat)].mean()
    average_s = s_mat[np.nonzero(s_mat)].mean()
    average_v = v_mat[np.nonzero(v_mat)].mean()
    return number_of_nonblackpixels,average_h,average_s,average_v

def getArclengthAreaRatio(img):
    number_of_nonblackpixels = np.count_nonzero(img[:,:,2])
    contours, hierarchy = cv.findContours(img[:,:,2],cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    total_arclength = 0
    for i in contours:
        total_arclength += cv.arcLength(i,True)

    return total_arclength/number_of_nonblackpixels

def whitePixelsAreaRatio(img):
    
    number_of_nonblackpixels = np.count_nonzero(img[:,:,2])
    return len(np.where(img[:,:,2]>250))/number_of_nonblackpixels

countours,c_sort_by_x,c_sort_by_y=contourprep(img3)







fire = getBaseWidthAndHeight(c_sort_by_y)
print(fire,getSymmetryScore(c_sort_by_x,fire["base_midpoint"]))
print('\n')
print(whitePixelsAreaRatio(img3),getArclengthAreaRatio(img3),getAttrributes(img3))
print('\n')

#getSymmetryScore(c_sort_by_x,fire["base_midpoint"])
# img3 = img3.astype(np.uint8)
# img3 = cv.drawContours(img3, c, -1, (0,255,0), 3)



{'width': 884, 'height': 483, 'base_midpoint': 470} 0.01941963458473854


1.1579032687609278e-05 0.09803821804714975 (172726, 16.917897938658534, 97.28264651150246, 170.20537151326377)




In [70]:
print(np.count_nonzero(img3),img3.shape[0]*img3.shape[1])


443341 518400


In [11]:
cv.imshow('Pred',img4)





cv.waitKey(0)
cv.destroyAllWindows()